In [1]:
import pandas as pd

In [2]:
df=messages = pd.read_csv('Dataset/SMSSpamCollection.txt', sep='\t',
                           names=["label", "message"])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.shape

(5572, 2)

In [4]:
X=list(df['message'])
y_raw=list(df['label'])
y=list(pd.get_dummies(y_raw,drop_first=True)['spam'])
y


[False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [6]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [7]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [8]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels,device='cpu'):
        self.encodings = encodings
        self.labels = labels
        self.device = device

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx],device=self.device) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long,device=self.device)
        return item

# Creating PyTorch datasets
train_dataset = CustomDataset(train_encodings, y_train,device)
test_dataset = CustomDataset(test_encodings, y_test,device)

# Example of using a DataLoader
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [20]:
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Load the model (PyTorch version)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# Define PyTorch training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    num_train_epochs=2,              # Total number of training epochs
    per_device_train_batch_size=8,   # Batch size per device during training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir="./logs",            # Directory for storing logs
    logging_steps=10,                # Logging frequency
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save model checkpoints at each epoch
    dataloader_pin_memory=False
)


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Assumes train_dataset is a PyTorch Dataset object
    eval_dataset=test_dataset,    # Assumes test_dataset is a PyTorch Dataset object
)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.002200,0.035914
2,0.000900,0.027479


TrainOutput(global_step=1116, training_loss=0.08525105582309826, metrics={'train_runtime': 1899.8026, 'train_samples_per_second': 4.692, 'train_steps_per_second': 0.587, 'total_flos': 548894189854416.0, 'train_loss': 0.08525105582309826, 'epoch': 2.0})

In [24]:
output=trainer.predict(test_dataset)[1]

In [25]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,output)

In [26]:
trainer.save_model('senti_model')